# Lab10: Language modelling


The exercise shows how a language model may be used to solve word-prediction tasks and used to generate text.

In [1]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [3]:
pd.set_option('display.max_colwidth', -1)

## Read the documentation of [Language modelling in the Transformers](https://huggingface.co/transformers/task_summary.html#language-modeling) library.

## Download three [Polish models](https://huggingface.co/models?filter=pl) from the Huggingface repository.

Pobrane zostały trzy modele:

* [`dkleczek/bert-base-polish-cased-v1`](https://huggingface.co/dkleczek/bert-base-polish-cased-v1)
* [`Geotrend/bert-base-pl-cased`](https://huggingface.co/Geotrend/bert-base-pl-cased)
* [`allegro/herbert-large-cased`](https://huggingface.co/allegro/herbert-large-cased)

In [4]:
class Model:
    
    def __init__(self, name, tokenizer, model):
        self.name = name
        self.tokenizer = tokenizer
        self.model = model

In [5]:
model_1 = Model(
    'dkleczek/bert-base-polish-cased-v1',
    AutoTokenizer.from_pretrained('dkleczek/bert-base-polish-cased-v1'),
    AutoModelForMaskedLM.from_pretrained('dkleczek/bert-base-polish-cased-v1')
)

Some weights of the model checkpoint at dkleczek/bert-base-polish-cased-v1 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model_2 = Model(
    'Geotrend/bert-base-pl-cased',
    AutoTokenizer.from_pretrained('Geotrend/bert-base-pl-cased'),
    AutoModelForMaskedLM.from_pretrained('Geotrend/bert-base-pl-cased')
)

In [7]:
model_3 = Model(
    'allegro/herbert-large-cased',
    AutoTokenizer.from_pretrained('allegro/herbert-large-cased'),
    AutoModelForMaskedLM.from_pretrained('allegro/herbert-large-cased')
)

Some weights of the model checkpoint at allegro/herbert-large-cased were not used when initializing BertForMaskedLM: ['cls.sso.sso_relationship.weight', 'cls.sso.sso_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
models = [model_1, model_2, model_3]

Przygotowana została funkcja pomocnicza niezbędna do tworzenia predykcji.

In [9]:
def make_predictions(model, sentence, number_of_predictions=5):
    sentence = sentence.replace('[MASK]', model.tokenizer.mask_token)
    encoded_sentence = model.tokenizer.encode(sentence, return_tensors='pt')

    mask_token_index = torch.where(encoded_sentence == model.tokenizer.mask_token_id)[1][0].item()
    token_logits = model.model(encoded_sentence)[0]
    mask_token_logits = token_logits[0, mask_token_index, :]
    
    return [
        sentence.replace(model.tokenizer.mask_token, model.tokenizer.decode([token]))
        for token
        in torch.topk(mask_token_logits, number_of_predictions)[-1]
    ]

## Produce the predictions for the following sentences (use each model and check 5 predictions):
* (M) Warszawa to największe [MASK].
* (D) Te zabawki należą do [MASK].
* (C) Policjant przygląda się [MASK].
* (B) Na środku skrzyżowania widać [MASK].
* (N) Właściciel samochodu widział złodzieja z [MASK].
* (Ms) Prezydent z premierem rozmawiali wczoraj o [MASK].
* (W) Witaj drogi [MASK].

In [31]:
sentences = [
    'Warszawa to największe [MASK].',
    'Te zabawki należą do [MASK].',
    'Policjant przygląda się [MASK].',
    'Na środku skrzyżowania widać [MASK].',
    'Właściciel samochodu widział złodzieja z [MASK].',
    'Prezydent z premierem rozmawiali wczoraj o [MASK].',
    'Witaj drogi [MASK].'
]

In [32]:
for sentence in sentences:
    results = pd.DataFrame()
    for model in models:
        results[model.name] = make_predictions(model, sentence)
    print(sentence)
    display(results)

Warszawa to największe [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Warszawa to największe miasto.,Warszawa to największe miasto.,Warszawa to największe miasto.
1,Warszawa to największe województwo.,Warszawa to największe miasta.,Warszawa to największe miasta.
2,Warszawa to największe lotnisko.,Warszawa to największe woj.,Warszawa to największe ..
3,Warszawa to największe miasteczko.,Warszawa to największe Warszawa.,Warszawa to największe lotnisko.
4,Warszawa to największe państwo.,Warszawa to największe miast.,Warszawa to największe miast.


Te zabawki należą do [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Te zabawki należą do ciebie.,Te zabawki należą do tzw.,Te zabawki należą do dzieci.
1,Te zabawki należą do mnie.,Te zabawki należą do pt.,Te zabawki należą do mnie.
2,Te zabawki należą do nas.,Te zabawki należą do pl.,Te zabawki należą do nas.
3,Te zabawki należą do pana.,Te zabawki należą do odc.,Te zabawki należą do najmłodszych.
4,Te zabawki należą do niego.,Te zabawki należą do ok.,Te zabawki należą do Ciebie.


Policjant przygląda się [MASK].


...

,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Właściciel samochodu widział złodzieja z bronią.,Właściciel samochodu widział złodzieja z pt.,Właściciel samochodu widział złodzieja z bronią.
1,Właściciel samochodu widział złodzieja z tyłu.,Właściciel samochodu widział złodzieja z tzw.,Właściciel samochodu widział złodzieja z bliska.
2,Właściciel samochodu widział złodzieja z ulicy.,Właściciel samochodu widział złodzieja z ul.,Właściciel samochodu widział złodzieja z kamerą.
3,Właściciel samochodu widział złodzieja z bliska.,Właściciel samochodu widział złodzieja z woj.,Właściciel samochodu widział złodzieja z nożem.
4,Właściciel samochodu widział złodzieja z zewnątrz.,Właściciel samochodu widział złodzieja z kościoła.,Właściciel samochodu widział złodzieja z kamery.


Prezydent z premierem rozmawiali wczoraj o [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Prezydent z premierem rozmawiali wczoraj o tym.,Prezydent z premierem rozmawiali wczoraj o północy.,Prezydent z premierem rozmawiali wczoraj o tym.
1,Prezydent z premierem rozmawiali wczoraj o Polsce.,Prezydent z premierem rozmawiali wczoraj o tym.,Prezydent z premierem rozmawiali wczoraj o sprawie.
2,Prezydent z premierem rozmawiali wczoraj o budżecie.,Prezydent z premierem rozmawiali wczoraj o nim.,Prezydent z premierem rozmawiali wczoraj o sytuacji.
3,Prezydent z premierem rozmawiali wczoraj o ASF.,Prezydent z premierem rozmawiali wczoraj o co.,Prezydent z premierem rozmawiali wczoraj o prywatyzacji.
4,Prezydent z premierem rozmawiali wczoraj o ustawie.,Prezydent z premierem rozmawiali wczoraj o św.,Prezydent z premierem rozmawiali wczoraj o ..


Witaj drogi [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Witaj drogi chłopcze.,Witaj drogi wyd.,Witaj drogi człowieku.
1,Witaj drogi przyjacielu.,Witaj drogi op.,Witaj drogi Panie.
2,Witaj drogi bracie.,Witaj drogi pt.,Witaj drogi Jezu.
3,Witaj drogi kolego.,Witaj drogi ps.,Witaj drogi panie.
4,Witaj drogi synu.,Witaj drogi gen.,Witaj drogi misiu.


### Komentarz

1. (M) **Warszawa to największe [MASK].**
    * wszystkie modele jako pierwszy wynik zwróciły zdanie `Warszawa to największe miasto.`
    * model `dkleczek` jako jedyny zachował poprawność gramatyczną, podczas gdy pozostałe modele zwróciły zdania typu `Warszawa to największe miasta.` czy `Warszawa to największe miast.`
2. (D) **Te zabawki należą do [MASK].**
    * modele `dkleczek` i `allegro` dały sensowne wyniki - `dkleczek` wstawił w miejsce maski różne osoby gramatyczne, a `allegro` uwzględniło też wyniki takie jak `dzieci` czy `najmłodszych`, które mają sens nie tylko pod względem gramatycznym, ale też semantycznym
    * warto zauważyć, że `allegro` zwróciło wyraz `Ciebie` pisany wielką literą, a `dkleczek` małą
    * modele `Geotrend` zwrócił dopasowania takie jak `tzw.` czy `ok.` - najwyraźniej potraktował kropkę nie jako znak kończący zdanie, a raczej oznaczenie skrótu
3. (C) **Policjant przygląda się [MASK].**
    * modele `dkleczek` i `allegro` zwróciły wyniki poprawne gramatycznie, ale wyniki `allegro` wydają mi się nieco ciekawsze (`sprawie`, `zdarzeniu`, `sytuacji`, podczas gdy `dkleczek` zwraca wyrazy takie jak `im` czy `temu`)
    * w przypadku `Geotrend` w przypadku części wyników wystąpił ten sam problem z interpunkcją, a reszta wyników jest raczej bez sensu (`Policjant przygląda się się`)
4. (B) **Na środku skrzyżowania widać [MASK].**
    * jak wyżej - zachowanie poprawności gramatycznej w przypadku modeli `allegro` i `dkleczek`
    * wyniki `allegro` dużo sensowniejsze semantycznie (`rondo`, `znak`, `samochód`) niż w przypadku modelu `dkleczek` (`rzekę`, `ulicę`, `las`)
    * wsród wyników `Geotrend` ponownie pojawiają się skróty, a z poprawnych gramatycznie `obraz`, `kościół`, `rok` - w sumie nie zdziwiłby mnie kościół na środku skrzyżowania w Polsce, jednak nie uznałabym tych wyników za wartościowe semantycznie
5. (N) **Właściciel samochodu widział złodzieja z [MASK].**
    * modele `dkleczek` i `allegro` ponownie wypadły z sensem - na pierwszym miejscu obydwa umieściły zdanie `Właściciel samochodu widział złodzieja z bronią.`
    * w modelu `allegro` wątpliwość budzi ewentualnie jedynie `Właściciel samochodu widział złodzieja z kamery.` (brzmi to jak zdanie, które ktoś mógłby wypowiedzieć, mając na myśli na przykład monitoring, ale chyba nie jest to do końca poprawne)
    * jedynym słowem w narzędniku zwróconym przez model `dkleczek` jest `broń`, a pozostałe określenia dotyczą tego, skąd właściciel samochodu widział złodzieja (`z tyłu`, `z zewnątrz`, `z ulicy`), podczas gdy model `allegro`  zwrócił więcej atrybutów złodzieja (`z kamerą`, `z nożem`), jednak nie jest to wada, bo modele miały prawo nie wiedzieć, że oczekujemy od nich narzędnika a nie dopełniacza   
    * te same problemy z `Geotrend` co wcześniej z wyjątkiem zdania `Właściciel samochodu widział złodzieja z kościoła.` (pytanie brzmi, czy to chodzi o ten kościół ze środka skrzyżowania z punktu wyżej)
6. (Ms) **Prezydent z premierem rozmawiali wczoraj o [MASK].**
    * modele `dkleczek` i `allegro` wypadły z sensem gramatycznie i semantycznie - poza `Prezydent z premierem rozmawiali wczoraj o ..` w przypadku `allegro`
    * model `Geotrend` zwrócił dwa błędne wyniki, dwa ogólniki (`o nim` i `o tym`) oraz jedną bardzo trafną obserwację polskiej rzeczywistości (`Prezydent z premierem rozmawiali wczoraj o północy.`)
7. (W) **Witaj drogi [MASK].**
     * sensowne wyniki modeli `dkleczek` i `allegro`
     * ciekawe zastosowanie wielkich liter w przypadku modelu `allegro` (`Witaj drogi Panie.` i `Witaj drogi panie.`)
     * w przypadku modelu `Geotrend` same skróty


Wyniki dla wołacza nieco się różnią po dodaniu przecinka po `Witaj`. Szczególnie ciekawe są wyniki modelu `Geotrend` - zawierają tajemnicze `##` wyglądające jak dla mnie na placeholder czy wildcard.

In [33]:
results = pd.DataFrame()
for model in models:
    results[model.name] = make_predictions(model, 'Witaj, drogi [MASK].')
print(sentence)
display(results)

Witaj drogi [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Witaj, drogi chłopcze.","Witaj, drogi ##arz.","Witaj, drogi człowieku."
1,"Witaj, drogi przyjacielu.","Witaj, drogi ##sz.","Witaj, drogi Panie."
2,"Witaj, drogi bracie.","Witaj, drogi ##er.","Witaj, drogi panie."
3,"Witaj, drogi panie.","Witaj, drogi ##ew.","Witaj, drogi Boże."
4,"Witaj, drogi kolego.","Witaj, drogi ##erk.","Witaj, drogi Jezu."


Co do wołacza, to wykonałam też analogiczny test dla rodzaju żeńskiego. W przypadku modelu `dkleczek` wyniki wydają mi się słabsze (`Witaj droga kochanie.`, `Witaj droga kochana.`). Model `allegro` wypada w porządku (ewentualnie `Witaj droga moja.` brzmi nieco niezręcznie). Model `Geotrend` zwraca bzdury przy braku przecinka, a po dodaniu przecinka wypada nieco lepiej, ale też nie porywa semantycznie. Po dodaniu przecinka w wynikach modeli `allegro` i `dkleczek` pojawia się urokliwy wynik w postaci zdania `Witaj, droga wolna`. Intryguje mnie też geneza zdania `Witaj, droga Betty.` - o ile geneza `Witaj droga Mario.` i `Witaj drogi Jezu.` jest dosyć jasna, to imię `Betty` mnie nieco zaskoczyło.

In [37]:
for sentence in [ 'Witaj droga [MASK].',  'Witaj, droga [MASK].']:
    results = pd.DataFrame()
    for model in models:
        results[model.name] = make_predictions(model, sentence)
    print(sentence)
    display(results)

Witaj droga [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Witaj droga pani.,Witaj droga ##mi.,Witaj droga Mario.
1,Witaj droga siostro.,Witaj droga ##ch.,Witaj droga moja.
2,Witaj droga mamo.,Witaj droga ##ł.,Witaj droga Polsko.
3,Witaj droga kochanie.,Witaj droga ##ń.,Witaj droga Pani.
4,Witaj droga kochana.,Witaj droga ##cji.,Witaj droga pani.


Witaj, droga [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Witaj, droga pani.","Witaj, droga polska.","Witaj, droga Mario."
1,"Witaj, droga siostro.","Witaj, droga polskie.","Witaj, droga moja."
2,"Witaj, droga wolna.","Witaj, droga woj.","Witaj, droga wolna."
3,"Witaj, droga Betty.","Witaj, droga r.","Witaj, droga pani."
4,"Witaj, droga dziewczyno.","Witaj, droga świata.","Witaj, droga Pani."


Rodzaj nijaki wypadł najsłabiej - jedynie `Witaj drogie słońce`, `Witaj drogie dziecko.` i `Witaj drogie auto.` (oraz chyba `Witaj drogie kochanie.`) są poprawne w kontekście powitania. Pozostałe wyniki wygądają jak dopasowanie liczby mnogiej (`drogie panie`, `drogie dzieci`, `drogie dziewczyny`), co może nawet być poprawne gramatycznie, jeśli `Witaj` jest rozkazem (nie ma to jednak sensu po dodaniu przecinka).


Na specjalną nagrodę za dobre chęci zasługuje `Witaj drogie drogi.` w przypadku modelu `Geotrend`.

In [38]:
for sentence in [ 'Witaj drogie [MASK].',  'Witaj, drogie [MASK].']:
    results = pd.DataFrame()
    for model in models:
        results[model.name] = make_predictions(model, sentence)
    print(sentence)
    display(results)

Witaj drogie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Witaj drogie panie.,Witaj drogie polskie.,Witaj drogie dziecko.
1,Witaj drogie dziecko.,Witaj drogie piosenki.,Witaj drogie słońce.
2,Witaj drogie kochanie.,Witaj drogie strony.,Witaj drogie panie.
3,Witaj drogie dziewczyny.,Witaj drogie drogi.,Witaj drogie dzieci.
4,Witaj drogie dzieci.,Witaj drogie słowa.,Witaj drogie auto.


Witaj, drogie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Witaj, drogie panie.","Witaj, drogie się.","Witaj, drogie dziecko."
1,"Witaj, drogie dziecko.","Witaj, drogie słowa.","Witaj, drogie panie."
2,"Witaj, drogie dziewczyny.","Witaj, drogie polskie.","Witaj, drogie dzieci."
3,"Witaj, drogie dzieci.","Witaj, drogie imię.","Witaj, drogie słońce."
4,"Witaj, drogie Panie.","Witaj, drogie drogi.","Witaj, drogie Panie."


## Check the model predictions for the following sentences (using each model):
* Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].
* Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK].

In [12]:
sentences = [
    'Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].',
    'Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK].'
]

In [13]:
for sentence in sentences:
    results = pd.DataFrame()
    for model in models:
        results[model.name] = make_predictions(model, sentence)
    print(sentence)
    display(results)

Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zgodził.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##wiedział.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie bał."
1,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie dowiedział.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie było.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie poddał."
2,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie martwił.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie stało.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zabił."
3,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie bał.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##dził.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie śmiał."
4,"Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zabił.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie ##dzi.","Gdybym wiedział wtedy dokładnie to co wiem teraz, to bym się nie zastanawiał."


Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zgodziła.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie było.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie bała."
1,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie bała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie ##wia.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zgodziła."
2,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie dowiedziała.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie stało.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie dowiedziała."
3,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zabiła.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie ##wiedział.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie zmieniła."
4,"Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie pojawiła.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie ##czyła.","Gdybym wiedziała wtedy dokładnie to co wiem teraz, to bym się nie śmiała."


### Komentarz

Modele `dkleczek` i `allegro` zachowują odpowiedni rodzaj oraz sens gramatyczny i semantyczny. Model `Geotrend` ponownie wypadł słabo (zwrócił znowu dużo wyników typu `##wiedział`, które wyglądają na placeholdery).

Z ciekawości wykonałam też test po dodaniu do zdania brakującego (na moje oko) przecinka. Nie zmieniło to znacząco wyników, ale miało wpływ na ich kolejność.

In [41]:
sentences = [
    'Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].',
    'Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].'
]

In [42]:
for sentence in sentences:
    results = pd.DataFrame()
    for model in models:
        results[model.name] = make_predictions(model, sentence)
    print(sentence)
    display(results)

Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zgodził.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie ##wiedział.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie bał."
1,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie bał.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie było.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie poddał."
2,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie dowiedział.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie stało.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie śmiał."
3,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie pojawił.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie ##dził.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zastanawiał."
4,"Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zabił.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie stanie.","Gdybym wiedział wtedy dokładnie to, co wiem teraz, to bym się nie zabił."


Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zgodziła.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie było.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie bała."
1,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie bała.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie ##wia.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zgodziła."
2,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie dowiedziała.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie stało.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie dowiedziała."
3,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie pojawiła.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie ##śli.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zmieniła."
4,"Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie zabiła.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie ##wiedział.","Gdybym wiedziała wtedy dokładnie to, co wiem teraz, to bym się nie śmiała."


## Check the model predictions for the following sentences:
* [MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.
* W wakacje odwiedziłem [MASK], który jest stolicą Islandii.
* Informatyka na [MASK] należy do najlepszych kierunków w Polsce.

In [14]:
sentences = [
    '[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.',
    'W wakacje odwiedziłem [MASK], który jest stolicą Islandii.',
    'Informatyka na [MASK] należy do najlepszych kierunków w Polsce.'
]

In [15]:
for sentence in sentences:
    results = pd.DataFrame()
    for model in models:
        results[model.name] = make_predictions(model, sentence)
    print(sentence)
    display(results)

[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Jego wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
1,"Mięso wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Za wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Krew wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
2,"Słońce wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Po wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
3,"Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Ogień wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."
4,"Piwo wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Ich wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.","Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza."


W wakacje odwiedziłem [MASK], który jest stolicą Islandii.


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,"W wakacje odwiedziłem kraj, który jest stolicą Islandii.","W wakacje odwiedziłem Island, który jest stolicą Islandii.","W wakacje odwiedziłem Oslo, który jest stolicą Islandii."
1,"W wakacje odwiedziłem Cypr, który jest stolicą Islandii.","W wakacje odwiedziłem Reykjavík, który jest stolicą Islandii.","W wakacje odwiedziłem Londyn, który jest stolicą Islandii."
2,"W wakacje odwiedziłem Meksyk, który jest stolicą Islandii.","W wakacje odwiedziłem Porto, który jest stolicą Islandii.","W wakacje odwiedziłem Liverpool, który jest stolicą Islandii."
3,"W wakacje odwiedziłem Gibraltar, który jest stolicą Islandii.","W wakacje odwiedziłem miasto, który jest stolicą Islandii.","W wakacje odwiedziłem Glasgow, który jest stolicą Islandii."
4,"W wakacje odwiedziłem Wellington, który jest stolicą Islandii.","W wakacje odwiedziłem Port, który jest stolicą Islandii.","W wakacje odwiedziłem Birmingham, który jest stolicą Islandii."


Informatyka na [MASK] należy do najlepszych kierunków w Polsce.


,dkleczek/bert-base-polish-cased-v1,Geotrend/bert-base-pl-cased,allegro/herbert-large-cased
0,Informatyka na wsi należy do najlepszych kierunków w Polsce.,Informatyka na uczelni należy do najlepszych kierunków w Polsce.,Informatyka na AGH należy do najlepszych kierunków w Polsce.
1,Informatyka na świecie należy do najlepszych kierunków w Polsce.,Informatyka na rynku należy do najlepszych kierunków w Polsce.,Informatyka na UW należy do najlepszych kierunków w Polsce.
2,Informatyka na żywo należy do najlepszych kierunków w Polsce.,Informatyka na internet należy do najlepszych kierunków w Polsce.,Informatyka na UJ należy do najlepszych kierunków w Polsce.
3,Informatyka na pewno należy do najlepszych kierunków w Polsce.,Informatyka na świecie należy do najlepszych kierunków w Polsce.,Informatyka na UAM należy do najlepszych kierunków w Polsce.
4,Informatyka na odległość należy do najlepszych kierunków w Polsce.,Informatyka na terenie należy do najlepszych kierunków w Polsce.,Informatyka na uczelni należy do najlepszych kierunków w Polsce.


### Komentarz

1. **[MASK] wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.**
    * `dkleczek` i `allegro` ponownie wypadły sensownie
        * na pierwszym miejscu zwróciły zdanie `Woda wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.` (model `allegro` zwrócił `wodę` nawet dwa razy)
        * `krew` i `piwo` też mogą mieć sens semantycznie, chociaż zakładam, że nie są prawdą (a wolę nie googlować temperatury wrzenia krwi)
        * semantycznie mniej sensu mają `ogień` i `mięso`, jednak gramatycznie są poprawne
        * ciekawy jest wynik `Nie wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.` zwrócony przez wszystkie trzy modele
    * model `Geotrend` ponownie wypadł słabo, choć zwrócone na pierwszym miejscu zdanie `Jego wrze w temperaturze 100 stopni, a zamarza w temeraturze 0 stopni Celsjusza.` pewnie ma sens w jakimś kontekście
2. **W wakacje odwiedziłem [MASK], który jest stolicą Islandii.**
    * najlepszą znajomością geografii wykazał się model `Geotrend` (zdanie `W wakacje odwiedziłem Reykjavík, który jest stolicą Islandii.` na drugim miejscu wśród wyników)
    * model `allegro` nie poradził sobie ani z geografią ani z gramatyką `W wakacje odwiedziłem Oslo, który jest stolicą Islandii.
    * model `dkleczek` zachował poprawność gramatyczną (chociaż całe życie byłam pewna, że `Wellington` ma rodzaj nijaki, jednak słownik mówi co innego)
3. **Informatyka na [MASK] należy do najlepszych kierunków w Polsce.**
* model `dkleczek` zachował poprawność gramatyczną, jednak semantycznie wypadł słabiej 
* model `allegro` jako jedyny udzielił prawdziwie rozsądnych odpowiedzi (`Informatyka na AGH należy do najlepszych kierunków w Polsce.`)
* model `Geotrend` zwrócił zaskakująco poprawne gramatycznie wyniki (`Informatyka na uczelni należy do najlepszych kierunków w Polsce.`)

## Answer the following questions:
### Which of the models produced the best results?
Powiedziałabym, że modele `allegro` i `dkleczek` dały najlepsze wyniki. Zdania zwracane przez `dkleczek` są czasem trochę bardziej "zachowawcze" (`Policjant przygląda się temu.`, `Te zabawki należą do niego.`), jednak unika on jednocześnie błędów, które popełnia czasem model `allegro` (`W wakacje odwiedziłem Oslo, który jest stolicą Islandii`). Zdania zwracane przez model `allegro` wydają mi się jednak bardziej wartościowe semantycznie (`Na środku skrzyżowania widać radiowóz.` zamiast `Na środku skrzyżowania widać rzekę.` czy `Informatyka na AGH należy do najlepszych kierunków w Polsce.` zamiast `Informatyka na wsi należy do najlepszych kierunków w Polsce.`). Model `allegro` zwrócił też kilka niekomletnych wyników takich jak `Prezydent z premierem rozmawiali wczoraj o ..`.

Model `Geotrend` miał wiele problemów z interpunkcją i traktował kropki jako sygnał, że powinien szukać skrótów do wstawienia w miejsce maski. Pojawiło się również sporo niekompletnych wyników zawierających metaoznaczenia (`Witaj droga ##mi.`). Zasługuje jednak na uznanie za znajomość stolicy Islandii.

### Was any of the models able to capture Polish grammar?
Wydaje mi się, że z gramatyką najlepiej uporał się model `dkleczek`. Można to zauważyć chociażby w przypadku zdania `Warszawa to największe [MASK].`. Model `allegro` wydawał się walczyć o poprawność semantyczną i zwrócił kilka form słowa `miasto`, podczas gdy model `dkleczek` zwrócił po prostu słowa takie jak `województwo` czy `państwo`, tworząc poprawne językowo konstrukcje. W przypadku zdania `W wakacje odwiedziłem [MASK], który jest stolicą Islandii.` wykorzystał jedynie słowa w rodzaju męskim.

### Was any of the models able to capture long-distant relationships between the words?
Modele `allegro` i `dkleczek` obydwa poradziły sobie ze zdaniem złożonym `Gdybym wiedział[a] wtedy dokładnie to, co wiem teraz, to bym się nie [MASK].` niezależnie od rodzaju podmiotu. Ciekawe było ich różne podejście do bardziej złożonych konstrukcji jak na przykład `Właściciel samochodu widział złodzieja z nożem.` i `Właściciel samochodu widział złodzieja z tyłu.` - `allegro` zwróciło więcej określeń opisujących złodzieja, a `dkleczek` więcej określeń opisujących sposób, w jaki był on widziany.

### Was any of the models able to capture world knowledge?
Moim zdaniem model `allegro` odzwierciedlił rzeczywistość dużo lepiej od modelu `dkleczek` (argumenty pojawiły się już wcześniej, patrz: AGH, radiowóz na skrzyżowaniu, zabawki należące do dzieci), ale znajomością geografii jako jedyny wykazał się model `Geotrend`. `Geotrend` zwrócił też kilka intrygujących w kontekście polskiej rzzeczywistości zdań takich jak `Na środku skrzyżowania widać kościół.` czy `Prezydent z premierem rozmawiali wczoraj o północy.`.

### What are the most striking errors made by the models?
* problem ze skrótami w przypadku `Geotrend`
* niekompletne wyniki z tokenem `##` w przypadku modelu `Geotrend`
* niekompletne zdania z `.` w miejsce maski w przypadku modelu `allegro`
* zdania bez sensu semantycznego zwracane przez `dkleczek` (`Warszawa to największe państwo.`)
* zdania bez sensu gramatycznego zwracane przez `allegro` (`Warszawa to największe miast.`)
* ogólny miszmasz w wynikach `Geotrend` ;)